In [1]:
import copy
import csv
import pandas as pd
import numpy as np
import scipy as scipy
from scipy.stats import pearsonr
import math
import matplotlib
import matplotlib.pyplot as plt
#from matplotlib_venn import venn3, venn3_circles, venn2, venn2_circles
import itertools
from copy import deepcopy
from collections import defaultdict
from datetime import datetime
import random
pd.options.display.mpl_style = 'default'
%matplotlib inline
import pickle
#%pylab inline
import os
import matplotlib.ticker as mtick
from scipy.stats import chi2_contingency
import re

In [39]:
from Bio import SeqIO

In [40]:
for record in SeqIO.parse('/project/voight_datasets/hg19/chr1.fa', "fasta"):
    print record.id

chr1


In [41]:
str(record.seq)[:10]

'NNNNNNNNNN'

In [5]:
import sys
sys.path.append('/home/oryoruk/aggarwala_voight_2016_code_data/substitution_rate_paper_code_and_data/helper_code/')
from modules import *
from function_wrapper import *

In [6]:
def get_seq_context_variant(chr_name,position,before,after):
    #Chr_name: chromosome name
    #position: position of variant
    #before after: nucleotides before or after the variant
    files = file_handles('/project/voight_datasets/hg19')
    chr_name = str(chr_name)
    position = int(position)
    before = int(before)
    after = int(after)
    str_around = files.fetch_str_var(chr_name,position,before,after) #string around the variant including it
    #print "sequence context "+str(before)+" basepairs before and "+str(after)+" after the variant including the middle position"+''.join(str_around)
    #return ''.join(str_around)
    return ''.join(str_around)

In [29]:
def get_seq_context_variant2(chr_name,position,before,after):
    return str(ref_genome_chr.seq)[position - (before+1):position + after].upper()

In [7]:
coord = 100000

# time? 

In [36]:
%%time
for coord in range(10000,11000):
    seq = get_seq_context_variant("1",coord,3,3)

CPU times: user 324 ms, sys: 1.13 s, total: 1.45 s
Wall time: 7.29 s


In [37]:
%%time
for coord in range(10000,11000):
    seq = str(record.seq)[coord-4:coord+3].upper()

CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 1.89 ms


In [38]:
%%time
for coord in range(10000,11000):
    seq = get_seq_context_variant2("1",coord,3,3)

CPU times: user 0 ns, sys: 3 ms, total: 3 ms
Wall time: 2.87 ms


# 3000 times faster

# how about accuracy?

In [33]:
chrom = '1'

In [34]:
ref_genome_chr = SeqIO.read('/project/voight_datasets/hg19/chr'+chrom+'.fa', "fasta")

In [35]:
no_of_different_pulls, no_of_identical_pulls = 0,0
for i in range(2500):
    coord = random.randint(10000,100000000)    
    if get_seq_context_variant("1",coord,3,3) != get_seq_context_variant2("1",coord,3,3):
        no_of_different_pulls += 1
    else:
        no_of_identical_pulls += 1
print no_of_identical_pulls, no_of_different_pulls

2500 0


# looks fine

# jobs

In [10]:
for chrom in ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","X"]:
    job_cmd = 'bsub -o ./nc_7mer_sub_rates_per_chrom_1kgp3/'+chrom+' -e ./nc_7mer_sub_rates_per_chrom_1kgp3/'+chrom+' -q voight_long python 7mer_sub_rates.py non_coding_all_after_pruned.interval_sorted_chr'+chrom+' AFR_SNPs_only_no_singletons.bed ./../../output/nc_7mer_sub_rates_per_chrom_1kgp3/  '+chrom
    print job_cmd

bsub -o ./nc_7mer_sub_rates_per_chrom_1kgp3/1 -e ./nc_7mer_sub_rates_per_chrom_1kgp3/1 -q voight_long python 7mer_sub_rates.py non_coding_all_after_pruned.interval_sorted_chr1 AFR_SNPs_only_no_singletons.bed ./../../output/nc_7mer_sub_rates_per_chrom_1kgp3/  1
bsub -o ./nc_7mer_sub_rates_per_chrom_1kgp3/2 -e ./nc_7mer_sub_rates_per_chrom_1kgp3/2 -q voight_long python 7mer_sub_rates.py non_coding_all_after_pruned.interval_sorted_chr2 AFR_SNPs_only_no_singletons.bed ./../../output/nc_7mer_sub_rates_per_chrom_1kgp3/  2
bsub -o ./nc_7mer_sub_rates_per_chrom_1kgp3/3 -e ./nc_7mer_sub_rates_per_chrom_1kgp3/3 -q voight_long python 7mer_sub_rates.py non_coding_all_after_pruned.interval_sorted_chr3 AFR_SNPs_only_no_singletons.bed ./../../output/nc_7mer_sub_rates_per_chrom_1kgp3/  3
bsub -o ./nc_7mer_sub_rates_per_chrom_1kgp3/4 -e ./nc_7mer_sub_rates_per_chrom_1kgp3/4 -q voight_long python 7mer_sub_rates.py non_coding_all_after_pruned.interval_sorted_chr4 AFR_SNPs_only_no_singletons.bed ./../../o

In [ ]:
for chrom in ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","19","20","21","22","X"]:
    for job_id in range(500):
        job_cmd = 'bsub -o ./nc_7mer_sub_rates_per_chrom_1kgp3/'+chrom+'/'JOB_ID -e ./nc_7mer_sub_rates_per_chrom_1kgp3/CHROM/JOB_ID -q voight_long python 7mer_sub_rates.py non_coding_all_after_pruned.interval_sorted_chrCHROM AFR_SNPs_only_no_singletons.bed ./../../output/nc_7mer_sub_rates_per_chrom_1kgp3/CHROM/ 500 JOB_ID